In [2]:
from gensim import utils
from gensim.models.doc2vec import LabeledSentence
from gensim.models import Doc2Vec

import numpy as np
from random import shuffle
from sklearn.linear_model import LogisticRegression

import logging
import os.path

program = os.path.basename(sys.argv[0])
logging.basicConfig(format="%(asctime)s: %(levelname)s : %(message)s",level=logging.INFO)
logger = logging.getLogger(program)
logger.info("running %s" % ' '.join(sys.agrv))

class LabeledLineSentence(object):
    
    def __init__(self,sources):
        self.sources = sources
        filpped = {}
        
        for key,value in sources.items():
            if value not in filpped:
                filpped[value] = [key]
            else:
                raise Exception("Non-unique prefix encountered")
    
    def __iter__(self):
        for source,prefix in self.sources.items():
            with utils.smart_open(source) as fr:
                for item_no,line in enumerate(fr):
                    yield LabeledSentence(utils.to_unicode(line).split(),[prefix + '_%s' % item_no])
    
    def to_array(self):
        self.sentences = []
        for source,prefix in self.sources.items():
            with utils.smart_open(source) as fr:
                for item_no,line in enumerate(fr):
                    self.sentences.append(LabeledSentence(utils.to_unicode(line).split(),[prefix + "_%s" % item_no]))
        return self.sentences
    
    def sentences_perm(self):
        shuffle(self.sentences)
        return self.sentences

sources = {"test-neg.txt":"TEST_NEG","test-pos.txt":"TEST_POS","train-pos.txt":"TRAIN_POS",
           "train-neg.txt":"TRAIN_NEG","train-unsup.txt":"TRAIN_UNS"}

sentences = LabeledLineSentence(sources)

model = Doc2Vec(min_count = 1,window = 6,size = 100,sample = 1e-3,negative =5 ,workers=8)
model.build_vocab(sentences.to_array())

for epoch in range(10):
    logger.info('Epoch %d' % epoch)
    model.train(sentences.sentences_perm())

model.save("./imdb.d2v")

/usr/lib/python2.7/dist-packages/sklearn/utils/sparsetools/__init__.py:3: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility
  from ._min_spanning_tree import minimum_spanning_tree
/usr/lib/python2.7/dist-packages/sklearn/utils/sparsetools/_graph_validation.py:5: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility
  from ._graph_tools import csgraph_to_dense, csgraph_from_dense,\
/usr/lib/python2.7/dist-packages/sklearn/utils/sparsetools/__init__.py:4: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility
  from ._traversal import connected_components
/usr/lib/python2.7/dist-packages/sklearn/utils/extmath.py:20: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility
  from ._logistic_sigmoid import _log_logistic_sigmoid
/usr/lib/python2.7/dist-packages/sklearn/utils/extmath.py:22: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility
  from .sparsefuncs_fast impor

NameError: name 'sys' is not defined

In [8]:
from gensim.models import Doc2Vec

sources = {"test-neg.txt":"TEST_NEG","test-pos.txt":"TEST_POS","train-pos.txt":"TRAIN_POS",
           "train-neg.txt":"TRAIN_NEG","train-unsup.txt":"TRAIN_UNS"}

model = Doc2Vec.load("./imdb.d2v")
print model.most_similar("good")

train_arrays = np.zeros((500,100))
train_labels = np.zeros(500)

for i in range(12500):
    prefix_train_pos = "TRAIN_POS_" + str(i)
    prefix_train_neg = "TRAIN_NEG_" + str(i)
    train_arrays[i] = model[prefix_train_pos]
    train_arrays[12500 + i] = model[prefix_train_neg]
    train_labels[i] = 1
    train_labels[12500 + i] = 0

print train_labels

[(u'nice', 0.9807448387145996), (u'nevertheless', 0.9732069969177246), (u'intelligent', 0.9726178646087646), (u'amusing', 0.9704580307006836), (u'hilarious', 0.9701518416404724), (u'great', 0.9691581726074219), (u'besides', 0.9689624905586243), (u'though', 0.9689328670501709), (u'quite', 0.9681524634361267), (u'very', 0.9669229984283447)]


KeyError: 'TRAIN_POS_0'